# PROJECT -  A2Z CUSTOMER SEGMENTATION

## Introduction
Data herein presented pertains to a fictional insurance company in Portugal, A2Z Insurance. The goal is to develop a customer segmentation strategy that will enable the Marketing Department to better understand the different customers' profiles and develop adequate marketing strategies. <br>
This project is done within the cope of the **Data Mining** curricular unit of the Master's Degree in **Data Science and Advanced Analytics**.

#### Group elements:
* Ivan Jure Parać (20210689)
* Nuno de Bourbon e Carvalho Melo (20210681)
* Stuart Gallina Ottersen (20210703)


## Table of Contents
1. [Data exploration](#data-exploration)
2. [Data preprocessing](#data-preprocessing)
    1. [First steps](#preprocessing-first-steps)
    2. [Dealing with outliers](#preprocessing-outliers)
    3. [Handling missing values](#preprocessing-missing-values)
    4. [Data transformation and cross field validation](#preprocessing-transform-validate)
    5. [Feature selection](#preprocessing-feature-selection)
    6. [Feature skewness and scaling](#preprocessing-scaling)
3. [Clustering](#data-clustering)
    1. [Sociodemographic clustering](#clustering-sociodemographic)

***

<h2><center>BOILERPLATE</center></h2>

***

In [ ]:
# uncomment next line of code to install package required for KPrototypes
# !pip install kmodes

In [ ]:
# import major libraries/modules
import pyreadstat
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sklearn.metrics as metrics

# others
from math import ceil
from regressors import stats
from scipy.cluster import hierarchy
from scipy.stats import chi2_contingency
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, AffinityPropagation, OPTICS, MeanShift
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

#from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load SAS file with the insurance company data
df, meta = pyreadstat.read_sas7bdat('a2z_insurance.sas7bdat')

# save copy of the original dataframe
original_df = df.copy()

<a class="anchor" id="data-exploration"></a>

***

<h2><center>DATA EXPLORATION</center></h2>

***

* First look at the dataset
* Set customer ID number as index
* Remove duplicate rows

In [ ]:
# first look at the dataframe
df.head()

In [ ]:
# check number of rows and columns
print("Number of observations:", df.shape[0])
print("Number of features:", df.shape[1])
print("Features:", list(df.columns))

In [ ]:
# describe the data
df.describe(include = "all").T

In [ ]:
# more information about the data
df.info()

<a class="anchor" id="data-preprocessing"></a>

***

<h2><center>DATA PREPROCESSING</center></h2>

***

<a class="anchor" id="preprocessing-first-steps"></a>

<h3><right>First steps</right></h3>

* Set customer ID as index
* Remove duplicated observations
* Convert EducDeg to float
* Swap incoherent birth and first policy years

In [ ]:
# set customer ID as index
df.CustID = df.CustID.astype("int")
df.set_index("CustID", inplace = True)

In [ ]:
# check for duplicated rows
print("Number of duplicates:", df.duplicated().sum())

# remove duplicate rows
df.drop_duplicates(inplace = True)
print("Removing duplicates...")
print("Number of duplicates:", df.duplicated().sum())

In [ ]:
# store initial number of rows (after duplicate removal)
initial_len = len(df)

# check number of rows and columns again
print("Number of observations:", df.shape[0])
print("Number of features:", df.shape[1])

df.head()

In [ ]:
# checking data types

# extract number from EducDeg, save as float
df.EducDeg = df.EducDeg.str.extract("(\d+)").astype("float")
# education degree mapper (to replace numbers when needed)
educ_mapper = {1: "Basic", 2: "High School", 3: "BSc/MSc", 4: "PhD"}

# check data types
df.dtypes

In [ ]:
# swaps years if policy seems to be made before birth
nr_incoherences = len(df[df.BirthYear > df.FirstPolYear])
pc_incoherences = round(nr_incoherences/len(df)*100, 1)

print(f"Number of people with a policy before birth: {nr_incoherences} "
    f"({pc_incoherences}% of the dataset)")

# such high number of inconsistencies suggests systematic error
# assumption: in these cases BirthYear and FirstPolYear were introduced in the wrong fields
print("Swapping birth and first policy years...")

# swap FirstPolYear and BirthYear values when birth occurs after first policy creation
row_filter = df.BirthYear > df.FirstPolYear
df.loc[row_filter, ["FirstPolYear", "BirthYear"]] = df.loc[row_filter, ["BirthYear", "FirstPolYear"]].values

# check if the years were correctly swapped
nr_incoherences = len(df[df.BirthYear > df.FirstPolYear])
print(f"Number of people with a policy before birth: {nr_incoherences}")

<a class="anchor" id="preprocessing-outliers"></a>

<h3><right>Dealing with outliers</right></h3>

* Visually inspect and remove outliers
* Store outliers in a separate dataframe

In [ ]:
def get_boxplots(df, features):

    sns.set()

    fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize = (20, 11))
    
    # iterate through axes objects and associate each box plot
    for ax, feat in zip(axes.flatten(), metric_features):
        sns.boxplot(x = df[feat], ax = ax)

    plt.show()

    return

In [ ]:
def remove_outliers(df):

    # BirthYear 1028 assumed to be a typo
    # 0 and 9 are fairly close in a qwerty keyboard, replaced with 1928
    df.loc[df.BirthYear == 1028, "BirthYear"] = 1928

    # conditions to remove outliers
    filters = ((df.FirstPolYear.ge(2017)),
            (df.MonthSal.ge(20000)),
            (df.CustMonVal.le(-2000)),
            (df.CustMonVal.ge(1500)),
            (df.ClaimsRate.ge(4)),
            (df.PremMotor.ge(3000)),
            (df.PremHousehold.ge(1600)),
            (df.PremHealth.ge(420)),
            (df.PremLife.ge(290)),
            (df.PremWork.ge(300)))

    # create a separate dataframe for the outliers
    df_outliers = pd.DataFrame()

    # remove outliers from main dataframe
    # add outliers to df_outliers
    for filter_ in filters:
        df_outliers = df_outliers.append(df[filter_])
        df = df[~filter_]
        
    # determine number of outliers removed
    n_outliers = len(df_outliers)
    pc_removed = round(n_outliers/initial_len*100, 2)
    print(f"Number of outliers removed: {n_outliers} ({pc_removed}% of all observations)")

    return (df, df_outliers)

In [ ]:
# define metric and non-metric features
metric_features = df.columns.drop(["EducDeg", "GeoLivArea", "Children"])
non_metric_features = ["EducDeg", "GeoLivArea", "Children"]

# boxplots of metric features
get_boxplots(df, metric_features)

In [ ]:
# remove outliers from df, store them in df_outliers
df, df_outliers = remove_outliers(df)

# boxplots of metric features after removing outliers
get_boxplots(df, metric_features)

In [ ]:
# new look at the data after removing outliers
df.describe(include = "all").T

<a class="anchor" id="preprocessing-missing-values"></a>

<h3><right>Handling missing values</right></h3>

* Check feature and row completeness (in df and df_outliers)
* Remove customers with no information about Premiums
* Remove customers with missing FirstPolYear or BirthYear
* Remove customers with missing EducDeg
* Fill Premium missing values with zero
* Create Linear Regression model to impute MonthSal
* Create Logistic Regression model to impute Children

In [ ]:
def check_completeness(df):

    # check feature completeness
    # number and percentage of NaN values per feature
    nr_nans = df.isna().sum()
    pc_nans = df.isna().mean()*100
    feature_nans = pd.concat([nr_nans, pc_nans], axis = 1)
    feature_nans.rename(columns = {0: "nr", 1: "%"}, inplace = True)

    # show number of missing values per feature
    print("Missing values per feature:\n", feature_nans)

    # check row completeness
    # max number of NaN values in a row and number of rows with that many NaN
    max_row_nan = df.isnull().sum(axis = 1).max()
    
    print(f"\nMaximum number of NaN values per row: {max_row_nan} "
        f"({len(df[df.isnull().sum(axis = 1) == max_row_nan])} rows)")

    return

In [ ]:
def remove_missing_values(df):

    # create dataframe to store rows with missing values discarded from the main dataframe
    df_nan = pd.DataFrame()

    # a row with 4 NaN (max number of missing values) has ~30% of its data missing (4 out of 13 features)
    # removing these rows - no information about Premiums
    max_nan_rows = df[df.isnull().sum(axis = 1) == df.isnull().sum(axis = 1).max()]
    df_nan = df_nan.append(max_nan_rows)
    df.drop(max_nan_rows.index, inplace = True)
    
    # remove rows with missing FirstPolYear and missing BirthYear
    firstpolyear_nan = df.FirstPolYear.isna()
    birthyear_nan = df.BirthYear.isna()
    df_nan = df_nan.append(df[firstpolyear_nan])
    df_nan = df_nan.append(df[birthyear_nan])
    df = df[~firstpolyear_nan]
    df = df[~birthyear_nan]

    # remove rows with missing EducDeg
    educ_nan = df.EducDeg.isna()
    df_nan = df_nan.append(df[educ_nan])
    df = df[~educ_nan]

    # make the function more verbose
    # show number of missing values removed
    nr_rows_removed = len(df_nan)
    total_removed = initial_len - len(df)

    print(f"Removed {len(max_nan_rows)} rows (~30% missing data, no information about Premiums)")
    print(f"Removed {sum(firstpolyear_nan)} rows (missing values in FirstPolYear)")
    print(f"Removed {sum(birthyear_nan)} rows (missing values in BirthYear)")
    print(f"Removed {sum(educ_nan)} rows (missing values in EducDeg)")
    print(f"Total number of rows removed: {nr_rows_removed}")
    print(f"Total number of rows removed so far: {total_removed} ({round(total_removed/initial_len*100, 2)}%)")

    return (df, df_nan)

In [ ]:
def impute_premiums(df):

    # replace NaN in Premiums with 0
    # assume no info about Premiums means no premium is paid
    motor_nan = df.PremMotor.isna()
    health_nan = df.PremHealth.isna()
    life_nan = df.PremLife.isna()
    work_nan = df.PremWork.isna()
    total_nan = motor_nan.sum() + health_nan.sum() + life_nan.sum() + work_nan.sum()

    df.PremMotor.fillna(0, inplace = True)
    df.PremHealth.fillna(0, inplace = True)
    df.PremLife.fillna(0, inplace = True)
    df.PremWork.fillna(0, inplace = True)

    # make the function verbose
    # show number of values imputed
    print(f"Imputed {sum(motor_nan)} values in PremMotor")
    print(f"Imputed {sum(health_nan)} values in PremHealth")
    print(f"Imputed {sum(life_nan)} values in PremLife")
    print(f"Imputed {sum(work_nan)} values in PremWork")
    print(f"Total number of imputations: {total_nan}")

    return df

In [ ]:
def impute_salaries(df, regressors):

    # independent, X, and dependent, y, variables
    X = df.dropna()[regressors]
    y = df.dropna().MonthSal

    # split train and validation data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 15)

    # reshaping needed if a string and not a list is passed as argument for regressors
    if type(regressors) is not list:
        X_train = np.array(X_train).reshape(-1, 1)
        X_val = np.array(X_val).reshape(-1, 1)

    # scale features using MinMaxScaler() with parameters from X_train
    scaler = MinMaxScaler().fit(X_train)
    # scale the training and test sets
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # create and fit model
    lin_reg = LinearRegression()
    lin_reg.fit(X_train_scaled, y_train)

    # predict salary of the validation set
    y_pred = lin_reg.predict(X_val_scaled)

    # compute metrics for the predictions made
    mse = metrics.mean_squared_error(y_val, y_pred)
    rmse = metrics.mean_squared_error(y_val, y_pred, squared = False)
    mae = metrics.mean_absolute_error(y_val, y_pred)

    print("")
    print("================================================")
    print("        MonthSal linear regression model        ")
    print("================================================")
    print("Mean square error:", round(mse, 2))
    print("Root mean square error:", round(rmse, 2))
    print("Mean absolute error:", round(mae, 2))
    stats.summary(clf = lin_reg, X = X_train_scaled, y = y_train)

    # predict MonthSal NaN values
    X_test = df.loc[df.MonthSal.isna(), regressors]

    # reshaping needed if a string and not a list is passed as argument for regressors
    if type(regressors) is not list:
        X_test = np.array(X_test).reshape(-1, 1)

    X_test_scaled = scaler.transform(X_test)
    y_pred = lin_reg.predict(X_test_scaled)
    print(y_pred)

    # impute values to MonthSal NaN
    print(f"\nImputed {len(y_pred)} values in MonthSal")
    #df.loc[df.MonthSal.isna(), "MonthSal"] = y_pred

    # multiple linear regression model to impute missing MonthSal values
    # use all features but the MonthSal to train the model

    # define independent and dependent variables
    #X = df.dropna().drop(["MonthSal"], axis = 1)
    #y = df.dropna().MonthSal

    # split train and test data
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15)

    # scale train and test data
    #scaler = MinMaxScaler().fit(X_train)
    #X_train_scaled = scaler.transform(X_train)
    #X_test_scaled = scaler.transform(X_test)

    # create and fit model
    #lin_model = LinearRegression()
    #lin_model.fit(X_train_scaled, y_train)

    # predict y
    #y_pred = lin_model.predict(X_test_scaled)

    # evaluate the predictions of the linear reg model
    #xlabels = X_train.columns
    #stats.summary(clf = lin_model, X = X_train_scaled, y = y_train, xlabels = xlabels)
    #mse = metrics.mean_squared_error(y_test, y_pred)
    #rmse = metrics.mean_squared_error(y_test, y_pred, squared = False)
    #mae = metrics.mean_absolute_error(y_test, y_pred)

    #print(mse)
    #print(rmse)
    #print(mae)

    #return df

In [ ]:
def impute_children(df, regressors):
            
    # imputing missing Children values
    # conclusion based on previous feature selection: use only BirthYear

    # independent, X, and dependent, y, variables
    X = df.dropna()[regressors]
    y = df.dropna().Children

    # split data into train (70%) and validation (30%) datasets
    # 70% have children, 30% dont, decided to stratify
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 5, stratify = y)

    # reshaping needed if a string and not a list is passed as argument for regressors
    if type(regressors) is not list:
        X_train = np.array(X_train).reshape(-1, 1)
        X_val = np.array(X_val).reshape(-1, 1)

    # scale features using MinMaxScaler() with parameters from X_train
    scaler = MinMaxScaler().fit(X_train)
    # scale the training and test sets
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # create a logistic regression model
    log_reg = LogisticRegression()
    log_reg.fit(X_train_scaled, y_train)

    # predict y
    y_pred = log_reg.predict(X_val_scaled)

    # evaluate the predictions of the logistic reg model
    conf_matrix = metrics.confusion_matrix(y_val, y_pred)
    conf_matrix = pd.DataFrame(conf_matrix)
    accuracy = round(metrics.accuracy_score(y_val, y_pred)*100, 2)
    precision = round(metrics.precision_score(y_val, y_pred)*100, 2)
    recall = round(metrics.recall_score(y_val, y_pred)*100, 2)
    f1 = round(metrics.f1_score(y_val, y_pred)*100, 2)

    print("========================================")
    print("   Children logistic regression model   ")
    print("========================================")
    print("Accuracy:", accuracy, "%")
    print("Precision:", precision, "%")
    print("Recall:", recall, "%")
    print("F1 score:", f1, "%")
    print("Confusion matrix:\n", conf_matrix)

    # predict Children NaN values
    X_test = df.loc[df.Children.isna(), regressors]

    # reshaping needed if a string and not a list is passed as argument for regressors
    if type(regressors) is not list:
        X_test = np.array(X_test).reshape(-1, 1)
    
    X_test_scaled = scaler.transform(X_test)
    y_pred = log_reg.predict(X_test_scaled)

    # impute values to Children NaN
    df.loc[df.Children.isna(), "Children"] = y_pred

    # attempting children prediction using KNN
    # overall results were worse than with logistic regression

    #data = df.dropna().drop(["Children", "GeoLivArea"], axis = 1)
    #target = df.dropna().Children

    #X_train, X_val, y_train, y_val = train_test_split(data, target, train_size=0.70, stratify = target, random_state=5)

    #modelKNN = KNeighborsClassifier()
    #modelKNN.fit(X = X_train, y = y_train)
    #labels_train = modelKNN.predict(X_train)
    #labels_val = modelKNN.predict(X_val)

    #conf_matrix = metrics.confusion_matrix(y_val, labels_val)
    #accuracy = round(metrics.accuracy_score(y_val, labels_val)*100, 2)
    #precision = round(metrics.precision_score(y_val, labels_val)*100, 2)
    #recall = round(metrics.recall_score(y_val, labels_val)*100, 2)
    #f1 = round(metrics.f1_score(y_val, labels_val)*100, 2)

    #print("Confusion matrix:\n", conf_matrix)
    #print("Accuracy:", accuracy, "%")
    #print("Precision:", precision, "%")
    #print("Recall:", recall, "%")
    #print("F1 score:", f1, "%") 

    return df

In [ ]:
# check missing values in the dataframe
check_completeness(df)

In [ ]:
# drops missing values
df, df_nan = remove_missing_values(df)

In [ ]:
# impute missing values in the Premium features
df = impute_premiums(df)

In [ ]:
# select feature(s) to use for linear regression of MonthSal
corr_salary = pd.Series(df.corr().MonthSal, name = "Correlation").sort_values()
print("Correlation between salary and other features:")
print(round(corr_salary, 3))

In [ ]:
# impute missing MonthSal values based on linear regression
salary_regressors = "BirthYear"
df = impute_salaries(df, salary_regressors)

In [ ]:
# check outliers dataframe for NaN values
outliers_nan_before = df_outliers.isna().sum()

# only 3 NaN - 1 PremMotor, 1 PremHealth, 1 PremLife
# assuming no info about premiums means no premium is paid
df_outliers.PremMotor.fillna(0, inplace = True)
df_outliers.PremHealth.fillna(0, inplace = True)
df_outliers.PremLife.fillna(0, inplace = True)

# check if NaN values were correctly imputed
outliers_nan_after = df_outliers.isna().sum()
outliers_nan = pd.concat([outliers_nan_before, outliers_nan_after], axis = 1)
outliers_nan.rename(columns = {0: "before", 1: "after"}, inplace = True)

print("Missing values in the outliers' dataframe:")
outliers_nan.T

In [ ]:
# selecting features for logistic regression of Children

# independent, X, and dependent, y, variables
X = df.dropna().drop(columns = "Children")
y = df.dropna().Children

# split data into train (70%) and validation (30%) datasets
# 70% have children, 30% dont, decided to stratify
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 5, stratify = y)

# scale features using MinMaxScaler() with parameters from X_train
scaler = MinMaxScaler().fit(X_train)
# scale the training set
X_train_scaled = scaler.transform(X_train)
# scale the test set
X_val_scaled = scaler.transform(X_val)

# recursive feature elimination
numfeats_list = np.arange(1, len(df.columns))
scores = {}

for n in range(len(numfeats_list)):
    log_reg = LogisticRegression()
    rfe = RFE(log_reg, numfeats_list[n])
    
    X_train_rfe = rfe.fit_transform(X_train_scaled, y_train)
    X_val_rfe = rfe.transform(X_val_scaled)
    log_reg.fit(X_train_rfe, y_train)
    
    score = log_reg.score(X_val_rfe, y_val)
    scores[n+1] = score

# RFE selected 1 single feature
best_num_feats = max(scores, key = scores.get)
rfe = RFE(estimator = log_reg, n_features_to_select = best_num_feats)
X_rfe = rfe.fit_transform(X = X_train_scaled, y = y_train)
selected_features = pd.Series(rfe.support_, index = X_train.columns, name = "RFE")

# compute correlation between Children and other features
corr_children = pd.Series(df.corr().Children, name = "Correlation")

# compute Lasso coefficients
reg = LassoCV()
reg.fit(X_train_scaled, y_train)
lasso_coef = pd.Series(reg.coef_, index = X_train.columns, name = "Lasso")

# concatenate features selected by rfe, correlations, and lasso coefficients
selection_df = pd.concat([selected_features, corr_children, lasso_coef], axis = 1).drop("Children")

# plot correlation and lasso coefficients
coef_names = ["Correlation", "Lasso"]

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, axes = plt.subplots(1, ceil(len(coef_names)), figsize = (24, 10))

for ax, coef in zip(axes.flatten(), coef_names):
    sns.barplot(data = selection_df,
                x = coef,
                y = selection_df.index,
                hue = "RFE",
                palette = ["darkgray", "C9"],
                order = selection_df.sort_values(coef).index,
                ax = ax)
    ax.axvline(x = 0, linestyle = "--", color = "darkgray", label = "_nolegend_")
    ax.set_xlabel(coef + " coefficient", fontsize = 16)
    ax.legend(title = "RFE", loc = "upper right", fontsize = 14)

plt.show()

In [ ]:
# impute children missing values based on selected features
children_regressors = ["BirthYear"]
df = impute_children(df, children_regressors)

In [ ]:
# check if all NaN values were dealt with
check_completeness(df)

<a class="anchor" id="preprocessing-transform-validate"></a>

<h3><right>Data transformation and cross field validation</right></h3>

* Add columns: Age, FirstPolAge, CustYears, Generation, PremTotal, Premium ratios
* Check if EducDeg makes sense given the customer's age
* Convert MonthSal to YearSal
* Remove outliers from newly created features
* Transform skewed metric features

In [ ]:
def feature_creator(df):
    
    # create Age feature
    curr_year = 2016
    cust_ages = curr_year - df.BirthYear
    df["Age"] = cust_ages
    
    # create FirstPolAge feature
    firstpol_ages = df.FirstPolYear - df.BirthYear
    df["FirstPolAge"] = firstpol_ages
    
    # create CustYears feature
    df["CustYears"] = curr_year - df.FirstPolYear
    
    # create a Generation feature
    df.loc[(df.BirthYear >= 1928) & (df.BirthYear <= 1945), "Generation"] = 1 # Silent Gen
    df.loc[(df.BirthYear >= 1946) & (df.BirthYear <= 1964), "Generation"] = 2 # Baby Boomer
    df.loc[(df.BirthYear >= 1965) & (df.BirthYear <= 1980), "Generation"] = 3 # Gen X
    df.loc[(df.BirthYear >= 1981) & (df.BirthYear <= 1996), "Generation"] = 4 # Millennial
    df.loc[(df.BirthYear >= 1997) & (df.BirthYear <= 2012), "Generation"] = 5 # Gen Z
    df.Generation = df.Generation.astype("float")
    
    # create a YearSal feature
    year_sals = df.MonthSal * 12
    df["YearSal"] = year_sals
    
    # create a PremTotal column
    premium_cols = ["PremMotor", "PremHousehold", "PremHealth", "PremLife", "PremWork"]
    df["PremTotal"] = df[premium_cols].sum(axis = 1)
    
    # create PremiumRatio columns
    for col in premium_cols:
        newcol_vals = df[col]/df["PremTotal"]
        newcol_name = col + "_ratio"
        df[newcol_name] = newcol_vals
        
    return df

In [ ]:
# update dataframe with new features
df = feature_creator(df)

In [ ]:
# get more information about the newly created Age, FirstPolAge, and CustYears features
year_newfeatures = ["Age", "FirstPolAge", "CustYears"]

# dataframe to store summary statistics
feats_info = pd.DataFrame()

# violin plots of these three features
sns.set(font_scale = 1.4)
sns.set_style("ticks")
fig, axes = plt.subplots(1, ceil(len(year_newfeatures)), figsize = (16, 4))

for ax, feat in zip(axes.flatten(), year_newfeatures):
    sns.violinplot(data = df,
                   x = df[feat],
                   color = "C9",
                   ax = ax)
    ax.set_xlim([-10, 110])
    
    # append summary statistics to feats_info dataframe
    feat_info = pd.Series(df[feat].describe())
    feats_info = feats_info.append(feat_info)

print("=============================================================================================================")
print("   Customer age (Age), age at first policy (FirstPolAge), and number of years with the company (CustYears)   ")
print("=============================================================================================================")
print(feats_info.T)

# almost 50% of the customers made their first policy before turning 18
pre_18_policy = len(df[df.FirstPolAge < 18])
print("\nMade first policy before 18:", round(pre_18_policy/len(df), 2)*100, "%")

In [ ]:
# cross-validate EducDeg and Age
# minimum age is 18 - meaning everyone can have an education up to High School
# minimum age of 20 for a BSc
# minimum age of 23 for a PhD (skipping MSc and finishing in 3 years, UK or outside of the EU)
educdeg_min_age = df.groupby("EducDeg").Age.min().rename(index = educ_mapper)
print("======================================")
print("   Cross-validating EducDeg and Age   ")
print("======================================")
print("Minimum age associated to each EducDeg")
print(educdeg_min_age)

# no incoherences found

In [ ]:
# visualize Generation feature
# generation mapper
gen_mapper = {1: "Silent Gen",
              2: "Baby Boomer",
              3: "Gen X",
              4: "Millennial",
              5: "Gen Z"}

# count number of customers per generation
gen_count = df.groupby("Generation").size().sort_values(ascending = False)
gen_count.rename(index = gen_mapper, inplace = True)

# visualize number of customers per generation
fig, ax = plt.subplots(figsize = (10, 6))
sns.barplot(x = gen_count.index, y = gen_count.values, order = gen_count.index)
plt.title("Number of customers per generation", fontsize = 22)
plt.xlabel("Generation", fontsize = 18)
plt.ylabel("Count", fontsize = 18)

plt.show()

# almost no customers from the younger generation, Gen Z

In [ ]:
# update metric and non-metric features
metric_features = df.columns.drop(["EducDeg", "GeoLivArea", "Children", "Generation"])
non_metric_features = ["EducDeg", "GeoLivArea", "Children", "Generation"]

# boxplot of current metric features
sns.set()

fig, axes = plt.subplots(5, ceil(len(metric_features) / 5), figsize = (20, 25))

# iterate through axes objects and associate each box plot
for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(x = df[feat], ax = ax)
    
plt.show()

In [ ]:
# remove outliers in newly created features?
# conditions to remove outliers
filters = ((df.PremHousehold_ratio.ge(0.85)),
           (df.PremHealth_ratio.ge(0.7)),
           (df.PremLife_ratio.ge(0.5)),
           (df.PremWork_ratio.ge(0.4)),
           (df.PremWork_ratio.le(-0.05)),
           (df.PremTotal.le(350)),
           (df.PremTotal.ge(1750)))

# number of observations before second round of outliers removal
nr_initial_outliers = len(df_outliers)

# remove outliers from main dataframe, store them in df_outliers
for filter_ in filters:
    df_outliers = df_outliers.append(df[filter_])
    df = df[~filter_]

nr_new_outliers = len(df_outliers) - nr_initial_outliers
total_removed = initial_len - len(df)
print(f"Number of outliers removed (2nd round): {nr_new_outliers}")
print(f"Total observations removed: {total_removed} ({round(total_removed/initial_len*100, 2)}%)")

In [ ]:
# boxplot of metric features after removing outliers
sns.set()

fig, axes = plt.subplots(5, ceil(len(metric_features) / 5), figsize = (20, 25))

# iterate through axes objects and associate each box plot
for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(x = df[feat], ax = ax)
    
plt.show()

<a class="anchor" id="preprocessing-feature-selection"></a>

<h3><right>Feature selection</right></h3>

* Remove redundant features
* Remove irrelevant features
* Prepare dataframe for cluster analysis

In [ ]:
# create a heatmap showing correlation between all features
plt.subplots(figsize = (15, 12))
mask = np.triu(np.ones_like(df.corr(), dtype = bool))
corr_heatmap = sns.heatmap(df.corr(), mask = mask, vmin = -1, vmax = 1, annot = True, cmap = 'BrBG')
corr_heatmap.set_title('Triangle Correlation Heatmap', fontdict = {'fontsize': 14}, pad = 8);

In [ ]:
# remove redundant features from main dataframe (perfect correlation)
df.drop(columns = ["FirstPolYear", "BirthYear", "MonthSal"], inplace = True)

# TO BE DECIDED (based on future clustering)
# Age vs FirstPolAge vs Generation vs YearSal
# ClaimsRate vs CustMonVal
# PremTotal vs PremHousehold
# PremTotal vs PremHousehold_ratio
# Prem ratios vs Prem values

In [ ]:
# create a heatmap showing correlation between updated features
plt.subplots(figsize = (15, 12))
mask = np.triu(np.ones_like(df.corr(), dtype = bool))
corr_heatmap = sns.heatmap(df.corr(), mask = mask, vmin = -1, vmax = 1, annot = True, cmap = 'BrBG')
corr_heatmap.set_title('Triangle Correlation Heatmap', fontdict = {'fontsize': 14}, pad = 8);

In [ ]:
# updating metric and non-metric features
metric_features = df.columns.drop(["EducDeg", "GeoLivArea", "Children", "Generation"])
non_metric_features = ["EducDeg", "GeoLivArea", "Children", "Generation"]

print("Metric features:", metric_features)
print("Non-metric features:", non_metric_features)

In [ ]:
# GeoLivArea has no meaningful correlation with any of the other features
# explore non-metric feature - GeoLivArea
sns.set()

fig, axes = plt.subplots(6, 3, figsize=(20, 25))

for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(x = df["GeoLivArea"], y = df[feat], ax = ax)
    
plt.show()

# GeoLivArea appears to have no discriminative power
# this was already hinted at in the previous correlation heatmap

In [ ]:
# remove GeoLivArea from the main dataframe
df.drop(columns = "GeoLivArea", inplace = True)

# remove GeoLivArea from the outliers dataframe
df_outliers.drop(columns = "GeoLivArea", inplace = True)

In [ ]:
# adding more outliers after creating features resulted in missing values for previous observations
nan_outliers_before = pd.Series(df_outliers.isna().sum(), name = "before")

df_outliers = feature_creator(df_outliers)

# check if missing values were dealt with
nan_outliers_after = pd.Series(df_outliers.isna().sum(), name = "after")
nan_outliers = pd.concat([nan_outliers_before, nan_outliers_after], axis = 1).T
print("Missing values in the outliers dataframe:")
display(nan_outliers)

# remove redundant features from outliers dataframe (based on corr heatmap)
df_outliers.drop(columns = ["FirstPolYear", "BirthYear", "MonthSal"], inplace = True)

In [ ]:
def restructure_df(df):
    
    # reorganise column order
    df = df.loc[:, ["Generation",
                   "Age",
                   "YearSal",
                   "EducDeg",
                   "Children",
                   "FirstPolAge",
                   "CustYears",
                   "CustMonVal",
                   "ClaimsRate",
                   "PremMotor",
                   "PremMotor_ratio",
                   "PremHousehold",
                   "PremHousehold_ratio",
                   "PremHealth",
                   "PremHealth_ratio",
                   "PremLife",
                   "PremLife_ratio",
                   "PremWork",
                   "PremWork_ratio",
                   "PremTotal"]]
    
    # rename a few columns
    df = df.rename(columns = {"CustMonVal": "CMV",
                             "PremMotor": "Motor",
                             "PremMotor_ratio": "MotorRatio",
                             "PremHousehold": "House",
                             "PremHousehold_ratio": "HouseRatio",
                             "PremHealth": "Health",
                             "PremHealth_ratio": "HealthRatio",
                             "PremLife": "Life",
                             "PremLife_ratio": "LifeRatio",
                             "PremWork": "Work",
                             "PremWork_ratio": "WorkRatio"})
    
    return df


# restructure and visualize the outliers dataframe before cluster analysis
df_outliers = restructure_df(df_outliers)
df_outliers.head()

In [ ]:
# restructure and visualize the main dataframe before cluster analysis
df = restructure_df(df)
df.head()

In [ ]:
# update metric and non-metric features
metric_features = df.columns.drop(["EducDeg", "Children", "Generation"])
non_metric_features = ["EducDeg", "Children", "Generation"]

print(f"Final number of observations: {df.shape[0]} ({round(df.shape[0]/initial_len*100, 2)}% of the original dataset)")
print(f"Current number of features: {df.shape[1]}")
print("Features:")
print(list(df.columns))

<a class="anchor" id="preprocessing-scaling"></a>

<h3><right>Feature skewness and scaling</right></h3>

In [ ]:
# dealing with skewed metric features
skewed_metric_feats = ["CMV",
                       "House",
                       "HouseRatio",
                       "Health",
                       "HealthRatio",
                       "Life",
                       "LifeRatio",
                       "Work",
                       "WorkRatio",
                       "PremTotal"]

# histograms of skewed metric features
sns.set()

fig, axes = plt.subplots(2, ceil(len(skewed_metric_feats) / 2), figsize = (22, 10))

# iterate through axes objects and associate each box plot
for ax, feat in zip(axes.flatten(), skewed_metric_feats):
    sns.histplot(x = df[feat], ax = ax)
    
plt.show()

In [ ]:
# storing skewed metric features before transformation for later
df_transformed = df.copy()

# add minimum value of skewed feature to each observation to ensure all values are non-negative
# then apply square root transformation
for feat in skewed_metric_feats:
    df_transformed[feat] = np.sqrt(df_transformed[feat] + abs(df_transformed[feat].min()))
    
# histograms of skewed metric features after sqrt transformation
sns.set()

fig, axes = plt.subplots(2, ceil(len(skewed_metric_feats) / 2), figsize = (22, 10))

# iterate through axes objects and associate each box plot
for ax, feat in zip(axes.flatten(), skewed_metric_feats):
    sns.histplot(x = df_transformed[feat], ax = ax)
    
plt.show()

In [ ]:
# list of features in the dataframe
features = list(df_transformed.columns)

# scaled version of the main dataframe
df_scaled = df_transformed.copy()
scaler = StandardScaler()
df_scaled[features] = scaler.fit_transform(df_transformed)

In [ ]:
# at this point there are 3 important dataframes

# dataframe containing the outliers
# exists in case we want to assign the outliers to clusters after the model is built
df_outliers.describe().T

In [ ]:
# the main dataframe
# used after obtaining cluster to make interpretation easier
df.describe().T

In [ ]:
# standardized dataframe
# obtained with StandardScaler after square rooting skewed features
# fed into a specific algorithm to obtain clusters
df_scaled.describe().T

<a class="anchor" id="data-clustering"></a>

***

<h2><center>CLUSTERING</center></h2>

***

<a class="anchor" id="clustering-sociodemographic"></a>

<h3><right>Sociodemographic clustering</right></h3>

In [ ]:
def elbow_kmeans(df, nmax_clusters):
    
    # store inertia values in a list
    inertia_vals = []
    
    # determine inertia for each number of clusters
    for n in np.arange(2, nmax_clusters + 1):
        km_clusters = KMeans(n_clusters = n, random_state = 15)
        km_clusters.fit(df)
        inertia = km_clusters.inertia_
        inertia_vals.append(inertia)
    
    # plot elbow graph
    sns.set_style("ticks")
    plt.subplots(figsize = (8, 6))
    sns.lineplot(x = np.arange(2, nmax_clusters + 1),
                 y = inertia_vals,
                 color = "k",
                 marker = 'o',
                 mew = 0,
                 linewidth = 3)
    plt.title("Elbow plot", fontsize = 22)
    plt.xlabel("Number of clusters, k", fontsize = 18)
    plt.ylabel("Inertia", fontsize = 18)
    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
    plt.xlim([0, nmax_clusters+1])
    
    return

In [ ]:
# sociodemographic features - Generation, Age, YearSal, EducDeg, Children
# YearSal redundant with Age
# Generation redundant with Age

# dataframe with sociodemographic features for KMeans
df_sociodem = df_scaled[["Age", "EducDeg", "Children"]]

df_sociodem.head()

In [ ]:
# elbow plot indicates ~7 clusters
elbow_kmeans(df_sociodem, 15)

In [ ]:
# init and fit KMeans model
km1_clusters = KMeans(n_clusters = 7, random_state = 15)
km1_clusters.fit(df_sociodem)

# get cluster labels
km1_labels = km1_clusters.labels_

In [ ]:
# inspect the differences between clusters

def describe_clusters(df, labels, features_list):
    
    # assign a label to each row in the dataframe
    df["Cluster"] = labels
    
    # get summary statistics of feature(s) passed on as argument(s)
    for feat in features_list:
        cluster_desc = pd.DataFrame()
        
        for label in set(labels):
            # compute statistics for each individual cluster
            cluster_feat_desc = pd.Series(round(df[df.Cluster == label][feat].describe()
                                                [["mean", "min", "50%", "max", "count"]], 2),
                                          name = "cluster" + str(label))
            
            # append statistics to cluster_desc dataframe
            cluster_desc = cluster_desc.append(cluster_feat_desc)

        print("=========================================")
        print("  " + feat)
        print("=========================================")
        print(cluster_desc.sort_values("mean"))
        
    return


describe_clusters(df, km1_labels, df_sociodem.columns)

In [ ]:
# strip plot for cluster visualization?
sns.set_style("ticks")

# test = df.copy()
# test.EducDeg = test.EducDeg.replace(educ_mapper)

x_vars = ["Age", "EducDeg"]

g = sns.PairGrid(df.sort_values("Age", ascending = False),
                 x_vars = x_vars, y_vars = ["Cluster"],
                 hue = "Children",
                 height = 8,
                 aspect = 0.85)

g.map(sns.stripplot, size = 12, orient = "h", jitter = True,
      palette = ["gray", "palevioletred"], alpha = 0.1)

# add better labels
g.set(ylabel = "Clusters")

titles = ["Age", "EducDeg"]

for ax, title in zip(g.axes.flat, titles):

    # set axis titles
    ax.set(title = title)

    # horizontal grid
    ax.xaxis.grid(False)
    ax.yaxis.grid(True, lw = 1.5)
    
plt.legend(title = "Children");

In [ ]:
# scatterplot for cluster visualization?
mean_cluster_ages = df.groupby("Cluster").Age.mean()
mean_cluster_educ = df.groupby("Cluster").EducDeg.mean()
mean_cluster_child = df.groupby("Cluster").Children.mean()
cluster_sizes = df.groupby("Cluster").size()

sns.set_style("ticks")
plt.subplots(figsize = (14, 8))
sns.scatterplot(x = mean_cluster_ages,
               y = mean_cluster_educ,
               hue = mean_cluster_child,
               palette = ["gray", "palevioletred"],
               s = cluster_sizes*10,
               alpha = 0.8,
               linewidth = 0)
plt.xlim([20, 80])
plt.xticks(np.arange(20, 81, 5))
plt.ylim([1, 4])
plt.yticks(np.arange(1, 5, 1));

In [ ]:
# investigate the remaining cluster features
df.groupby("Cluster").mean().sort_values("Age")

**Summary of the sociodemographic clusters obtained**

|             | C1    | C2    | C3    |          C4 |          C5 |      C6 |      C7 |
| -----       | ----- | ----- | ----- | ----------- | ----------- | ------- | ------- |
| **Age**     | Young | Young | Young | Middle-aged | Middle-aged | Elderly | Elderly |
| **Educ**    | Low   | Low   | High  | Low         | High        | Low     | High    |
| **Chilren** | Yes   | No    | Yes   | Yes         | Yes         | No      | No      |

C1 - Young uneducated adults with children <br>
C2 - Young uneducated adults without children <br>
C3 - Young educated adults with children <br>
C4 - Middle-aged uneducated adults with children <br>
C5 - Middle-aged educated adults with children <br>
C6 - Elderly with low education and no children <br>
C7 - Elderly with high education and no children

## Sociodemographic Clustering

For sociodemographic clustering, we used the following algorithms (and features):
1. KPrototypes (Age/Generation, EducDeg, Children)
2. Agglomerative Clustering (Age/Generation, EducDeg, Children)
3. Agglomerative Clustering (Age/Generation, EducDeg)
4. KMeans (Age/Generation, EducDeg, Children)
5. KMeans ((Age/Generation, EducDeg)

Tried DBScan and Mean Shift but because they are density based the weight they give to the Children binary value makes it so that only two clusters are found. Discarding Children and using only Age and EducDeg did not lead to better solutions.

Agglomerative Clustering and KMeans, because they are distance-based clustering algorithms, end up giving a lot of weight to binary variables, in this case Children. When analysing the results and the previous box plots, we concluded that while presence or absence of Children had some impact in a couple of the Premiums, that did not justify the weight they were receiving using these algorithms. For that reason, we tried both of them after discarding the feature Children. However, we felt uncomfortable losing Children as the boxplot suggests that it affects how much people pay for their Health insurance, which is the only Premium that does not appear to significantly change with EducDeg and it is not at all correlated to Age.

We opted with KMeans (k = 7) as even though it attributes significant weight to children (being a binary variable), the results are similar to KPrototypes (k = 6), with the added advantage of allowing the identification of the sociodemographic group with the highest CMV by far. So overweighing children did not appear to bias the final conclusions. Plus, KPrototypes is excruciantingly slow.

In KPrototypes, we tried 3 approaches:
1. Create a high number of clusters (15) and agglomerate based on distance (as determined via a dendrogram)
2. Create 4 clusters directly
3. Create 6 clusters directly

In [ ]:
# delete this cell if not using hierarchical clustering

def hierarchical_clusters(df, n_clusters = 2, threshold = None, affinity = "euclidean", linkage = "ward"):
    
    # determine clusters
    clusters = AgglomerativeClustering(n_clusters = n_clusters,
                                       affinity = affinity,
                                       linkage = linkage,
                                       distance_threshold = threshold)
    clusters.fit(df)
    
    # retrieve cluster labels and distances
    labels = clusters.labels_
    distances = clusters.distances_
    
    counts = np.zeros(clusters.children_.shape[0])
    n_samples = len(labels)
    
    for i, merge in enumerate(clusters.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count
    
    linkage_matrix = np.column_stack([clusters.children_, distances, counts]).astype(float)
    
    return (labels, distances, linkage_matrix)

In [ ]:
# automate categorical feature detection

def elbow_kprototypes(df, nmax_clusters):
    
    n_clusters = np.arange(2, nmax_clusters+1)
    cost_vals = []
    
    for n in n_clusters:
        kp_clusters = KPrototypes(n_clusters = n, init = "Huang", random_state = 15)
        kp_clusters.fit(df, categorical = [2])
        cost = kp_clusters.cost_
        cost_vals.append(cost)
        
    plt.subplots(figsize=(8, 8))
    sns.lineplot(x = np.arange(2, nmax_clusters + 1), y = cost_vals)

___
___
KP 1 - Age, EducDeg, Children (k = 4)

In [ ]:
#kp1_sociodem = pd.concat([df_scaled[["Age", "EducDeg"]], df["Children"]], axis = 1)

In [ ]:
#elbow_plot(kp1_sociodem, 15)

In [ ]:
#kp1_clusters = KPrototypes(n_clusters = 4, init = "Huang", random_state = 15)
#kp1_clusters.fit(kp1_sociodem, categorical = [2])
#kp1_labels = kp1_clusters.labels_
#kp1_centroids = kp1_clusters.cluster_centroids_
#kp1_linkage = hierarchy.linkage(kp1_centroids, method = "ward")

In [ ]:
#hierarchy.dendrogram(kp1_linkage, color_threshold = 2.0);

In [ ]:
#df["Cluster"] = kp1_labels
#df.groupby("Cluster").mean().sort_values("Age")

___
___
KP2 - Age, EducDeg, Children (k = 6)

In [ ]:
#kp2_clusters = KPrototypes(n_clusters = 6, init = "Huang", random_state = 15)
#kp2_clusters.fit(kp1_sociodem, categorical = [2])
#kp2_labels = kp2_clusters.labels_
#kp2_centroids = kp2_clusters.cluster_centroids_

In [ ]:
#kp2_linkage = hierarchy.linkage(kp2_centroids, method = "ward")
#hierarchy.dendrogram(kp2_linkage, color_threshold = 2.0);

In [ ]:
#df["Cluster"] = kp2_labels
#df.groupby("Cluster").mean().sort_values("Age")

___
___
KMeans 2 - Age, EducDeg, Children

KMeans + Hierarchical Clustering produced very similar results, so might as well go with this (simpler) approach.